In [1]:
#!pip install findspark
#!pip install pandas
#!pip install pyspark

In [2]:
import pyspark

#Crear una sesión de Spark
from pyspark.sql import SparkSession

#Importar funciones y tipos
from pyspark.sql.functions import *
#from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

from pyspark.sql.window import Window
from pyspark.sql import functions as f

#Crear el objeto Spark session
spark = SparkSession.builder.appName("BD12").getOrCreate()


In [3]:
##### CARGO LOS DATASETS

#Leer el archivo de disco --------------->> SE DEBE AJUSTAR LA RUTA

dfOriginal2021 = spark.read.option("header","true").csv("C:/Datasets/world-happiness-report-2021.csv")

dfOriginalAll = spark.read.option("header","true").csv("C:/Datasets/world-happiness-report.csv")



In [4]:
# Visualizacion de los shemas de los DF

dfOriginal2021.printSchema()
dfOriginalAll.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained 

In [5]:
#
# Ejercicio 1 - ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” 
#               mayor número más feliz es el país)

In [6]:
# Selecciono las columnas a utilizar
dfWork1 = dfOriginal2021.select(col("Country name").alias("CountryName"),
                            col("Regional indicator").alias("RegionalIndicator"),
                            col("Ladder score").cast(DoubleType()).alias("LadderScore"))

# Examino el esquema generado
#dfWork1.printSchema()

# Obtento el dato solicitado
dfPunto1 = dfWork1.orderBy(col("LadderScore").desc()).first()

# Muestro el resultado
print("\033[1mEl pais con mayor indice de felicidad en 2021 es:\033[0m")
print("Pais  :", dfPunto1["CountryName"])
print("Score :", dfPunto1["LadderScore"])


El pais con mayor indice de felicidad en 2021 es:
Pais  : Finland
Score : 7.842


In [7]:
#
# Ejercicio 2 - ¿Cuál es el país más “feliz” del 2021 por continente según la data
#

In [8]:
# Continuo trabajando en el mismo DF, genero las agrupaciones y datos necesarios
dfPunto2 = dfWork1.groupBy("RegionalIndicator", "CountryName").agg(f.max("LadderScore").alias("MaxLadderScore"))

# Calculo el maximo Ladder por Region
window_region = Window.partitionBy("RegionalIndicator")
dfMax_Ladder_Score = dfPunto2.withColumn("LadderScore", f.max("MaxLadderScore").over(window_region ))


# Filtro por el Max y selecciono las filas                               
dfPunto2 = dfMax_Ladder_Score.filter(f.col("LadderScore") == f.col("MaxLadderScore"))
dfPunto2 = dfPunto2.select("RegionalIndicator", "CountryName", "LadderScore")

# Preparo los resultados a mostrar
print("\033[1mLos paises mas felices por continente del 2021 son:\033[0m")

dfPunto2 = dfPunto2.withColumnRenamed("RegionalIndicator", "Regional Indicator") \
                    .withColumnRenamed("CountryName", "Country Name") \
                    .withColumnRenamed("LadderScore", "Ladder Score")

# Depliego con Pandas para mejor definicion
display(dfPunto2.toPandas())


Los paises mas felices por continente del 2021 son:


,Regional Indicator,Country Name,Ladder Score
0,Central and Eastern Europe,Czech Republic,6.965
1,Commonwealth of Independent States,Uzbekistan,6.179
2,East Asia,Taiwan Province of China,6.584
3,Latin America and Caribbean,Costa Rica,7.069
4,Middle East and North Africa,Israel,7.157
5,North America and ANZ,New Zealand,7.277
6,South Asia,Nepal,5.269
7,Southeast Asia,Singapore,6.377
8,Sub-Saharan Africa,Mauritius,6.049
9,Western Europe,Finland,7.842


In [9]:
#
# Ejercicio 3 - ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
#

In [10]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window


# Selecciono las columnas a utilizar
dfWork3 = dfOriginalAll.select( col("Country name").alias("CountryName"),
                                col("year").cast(IntegerType()).alias("Year"), 
                                col("Life Ladder").cast(DoubleType()).alias("LifeLadder"))

# Selecciono, creo y adecuo las columnas a utilizar del dataest 2021
dfWork3a = dfOriginal2021.select( col("Country name").alias("CountryName"), col("Ladder score").cast(DoubleType()).alias("LifeLadder"))
dfWork3a = dfWork3a.withColumn("Year",lit(2021))
dfWork3a = dfWork3a.select("CountryName", "Year", "LifeLadder")

#dfWork3.show()
#dfWork3a.show()

# Uno los DF
dfWork3 = dfWork3.union(dfWork3a)


# Creo las particiones para numerar las posiciones
WinYear = Window.partitionBy("Year").orderBy(F.desc("LifeLadder"))
dfWork3 = dfWork3.withColumn("Ranking", F.row_number().over(WinYear))

# Selecciono los paises que tuvvieron la posicion 1
dfPosiciones1 = dfWork3.filter(F.col("Ranking") == 1)


# Agrupo y cuento las posiciones 1 que tuvo cada pais
dfResultado = dfPosiciones1.groupBy("CountryName").agg(F.count("*").alias("Ranking1Count"))

# Ordeno el resultado descendente.
dfResultado = dfResultado.orderBy(F.desc("Ranking1Count"))

# Selecciono el pais con mas veces la posicion 1
Mas_Frecuente_Pos1 = dfResultado.first()

dfResultado.show()





+-----------+-------------+
|CountryName|Ranking1Count|
+-----------+-------------+
|    Finland|            7|
|    Denmark|            7|
|     Norway|            1|
|Switzerland|            1|
|     Canada|            1|
+-----------+-------------+



In [11]:
#
# Ejercicio 4 - ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?
#

In [12]:
#Selecciono las columnas a utilizar
dfWork4 = dfOriginalAll.select( col("Country name").alias("CountryName"),
                                col("year").cast(IntegerType()).alias("Year"), 
                                col("Life Ladder").cast(DoubleType()).alias("LifeLadder"),
                                col("Log GDP per capita").cast(DoubleType()).alias("GDP"))

# Creo las particiones para numerar las posiciones
WinYear = Window.partitionBy("Year").orderBy(F.desc("LifeLadder"))
dfWork4 = dfWork4.withColumn("Ranking", F.row_number().over(WinYear))

# Filtro por año 2020 y ordeno
dfResultado = dfWork4.filter(F.col("Year") == 2020).orderBy(F.desc("GDP"))

# Obtengo el mayor
Puesto_Felicidad_Pais_Mayor_GDP_2020 = dfResultado.first()

# Muestro el resultado
dfResultado.show(1)

print("\033[1mEl país con mayor GDP en 2020 es\033[0m", Puesto_Felicidad_Pais_Mayor_GDP_2020["CountryName"] , " \033[1my ocupa la posición de Felicidad :\033[0m", Puesto_Felicidad_Pais_Mayor_GDP_2020["Ranking"])


+-----------+----+----------+------+-------+
|CountryName|Year|LifeLadder|   GDP|Ranking|
+-----------+----+----------+------+-------+
|    Ireland|2020|     7.035|11.323|     13|
+-----------+----+----------+------+-------+
only showing top 1 row

El país con mayor GDP en 2020 es Ireland  y ocupa la posición de Felicidad : 13


In [13]:
#
# Ejercicio 5 - ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó 
#                o disminuyó?

In [14]:
#Selecciono las columnas a utilizar

# Respecto a los datos historicos
dfWork5_2020 = dfOriginalAll.select(col("year").cast(IntegerType()).alias("Year"), 
                                col("Log GDP per capita").cast(DoubleType()).alias("GDP"))

# Respecto a los datos 2021
dfWork5_2021 = dfOriginal2021.select(col("Logged GDP per capita").cast(DoubleType()).alias("GDP"))

# Genero las vistas para realizar las consultas
dfWork5_2020.createOrReplaceTempView("temp_table_2020")
dfWork5_2021.createOrReplaceTempView("temp_table_2021")

# Genero los DF con la consultas
df2020 = spark.sql("SELECT AVG(GDP) AVG_GDP_2020 FROM temp_table_2020 WHERE Year = 2020")
df2021 = spark.sql("SELECT AVG(GDP) AVG_GDP_2021 FROM temp_table_2021")

# Obtengo los valores
avg_2020 = df2020.select("AVG_GDP_2020").first()[0]
avg_2021 = df2021.select("AVG_GDP_2021").first()[0]

# Calculo la variación
porcentage_variacion = ((avg_2021 - avg_2020) / avg_2020) * 100
mayor_2021 = "aumento" if porcentage_variacion > 0 else 'disminuyo'
# Muestro el resultado
print("\033[1mEl porcentage de variación es : \033[0m", porcentage_variacion, "%")
print("\033[1mEl promedio del 2021 respecto al 2020 : \033[0m", mayor_2021)


El porcentage de variación es :  -3.2725946782511013 %
El promedio del 2021 respecto al 2020 :  disminuyo


In [15]:
#
# Ejercicio 6 - ¿Cuál es el país con mayor expectativa de vide (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia 
#                en ese indicador en el 2019?


In [16]:
#Selecciono las columnas a utilizar
dfWork6 = dfOriginalAll.select(col("Country name").alias("CountryName"),
                                col("year").cast(IntegerType()).alias("Year"), 
                                col("Healthy life expectancy at birth").cast(DoubleType()).alias("LifeExpectancy"))



# Selecciono, creo y adecuo las columnas a utilizar del dataest 2021
dfWork6_2021 = dfOriginal2021.select( col("Country name").alias("CountryName"), col("Healthy life expectancy").cast(DoubleType()).alias("LifeExpectancy"))
dfWork6_2021 = dfWork6_2021.withColumn("Year",lit(2021))
dfWork6_2021 = dfWork6_2021.select("CountryName", "Year", "LifeExpectancy")

#dfWork3.show()
#dfWork3a.show()

# Uno los DF
dfWork6 = dfWork6.union(dfWork6_2021)




# Calculo el pais con mayor espectativa de vida
dfMayor_Espectativa = dfWork6.select("CountryName").orderBy(F.desc("LifeExpectancy"))
Mayor_Espectativa = dfMayor_Espectativa.first()[0]


# Filtron por el año solicitado y el nombre de pais obtenido como con mayor espectativa
dfMayor_Espectativa_2019 =  dfWork6.filter( (F.col("Year") == 2019) & (F.col("CountryName") == Mayor_Espectativa) )
Mayor_Espectativa_2019 = dfMayor_Espectativa_2019.first()[0]


# Muestro el resultado
print("\033[1mEl pais con mayor espectativa de vida historicamente es : \033[0m", Mayor_Espectativa)

dfMayor_Espectativa.show(1)

dfMayor_Espectativa_2019.show()



El pais con mayor espectativa de vida historicamente es :  Singapore
+-----------+
|CountryName|
+-----------+
|  Singapore|
+-----------+
only showing top 1 row

+-----------+----+--------------+
|CountryName|Year|LifeExpectancy|
+-----------+----+--------------+
|  Singapore|2019|          77.1|
+-----------+----+--------------+

